In [4]:
#for performing HTTP requests
import requests

#for xml & html scrapping
from bs4 import BeautifulSoup

#url of wikipedia page from which I want to scrap tabular data.
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"


In [7]:
s = requests.Session()
response = s.get(url,timeout=10)
soup = BeautifulSoup(response.text,"html.parser")
response

<Response [200]>

In [9]:
import pandas as pd # library for data analsysis

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [14]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [17]:
import numpy as np

df["Latitude"] = np.nan
df["Longitude"] = np.nan
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,NaN,NaN
1,M4A,North York,Victoria Village,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",NaN,NaN
3,M6A,North York,"Lawrence Manor, Lawrence Heights",NaN,NaN
4,M7A,Queen's Park,Ontario Provincial Government,NaN,NaN
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",NaN,NaN
99,M4Y,Downtown Toronto,Church and Wellesley,NaN,NaN
100,M7Y,East Toronto Business,Enclave of M4L,NaN,NaN
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",NaN,NaN


In [23]:
! pip install geocoder

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [29]:
df2 = pd.read_csv ('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [32]:
for index, row in df.iterrows():
    postal_code = row['PostalCode']
    
    df3 = df2.loc[df2['Postal Code'] == postal_code]
    latitude = df3['Latitude']
    longitude = df3['Longitude']
    
    df.set_value(index, 'Latitude', latitude)
    df.set_value(index, 'Longitude', longitude)

df

AttributeError: 'DataFrame' object has no attribute 'set_value'